# Template del Prompt para el Trainig


In [1]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """\
### Instrucciones:
{system_message}

### Pregunta:
{input}

### Contexto:
{context}

### Respuesta:
<SQL>
{response}
</SQL>
"""


In [2]:
SYSTEM_MESSAGE = (
   "Eres un modelo especializado en convertir lenguaje natural a SQL."
   "Dada una pregunta en lenguaje natural y un contexto."
   "Responde solo **solo con la consulta SQL** correspondiente."
)

In [3]:
contexto="""
CREATE TABLE rpt_actual_casos (
	IdCaso INT NOT NULL,
    idTicket BIGINT NULL,
    idCliente BIGINT NULL,
    canal VARCHAR(45) NULL,
    estadoTicket INT NULL,
    Estado VARCHAR(50) NULL,
    frecepcion TIMESTAMP NULL,
    fgestion TIMESTAMP NULL,
    fechaEjecutivo TIMESTAMP NULL,
    fechaTicket TIMESTAMP NULL,
    fechaCierreRequerimiento TIMESTAMP NULL,
    tiempoRespuesta_min INT NULL,
    nombre_area VARCHAR(200) NULL,
    nombre_categoria VARCHAR(500) NULL,
    descripcionProceso VARCHAR(100) NULL,
    descripcionSubProceso VARCHAR(100) NULL,
    mensajeTicket TEXT NULL,
    scriptFase NVARCHAR(100) NULL,
    fechaAsignacion TIMESTAMP NULL,
    fechaResolucion TIMESTAMP NULL,
    mensajeResolucion TEXT NULL,
    nombreCliente VARCHAR(136) NOT NULL,
    descripcionGrupo VARCHAR(100) NULL,
    PNR VARCHAR(50) NULL,
    Usuario_Creacion VARCHAR(50) NULL,
    Usuario_Cierre VARCHAR(50) NULL
);
"""

In [4]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = contexto,
      response = sample["answer"]
  )

  return {"text" : full_prompt}

# Carga de Datos

In [5]:
!pip install datasets trl

In [6]:
from datasets import Dataset, DatasetDict

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
df=DatasetDict.load_from_disk('/content/drive/MyDrive/IA/sql_dataset')

In [9]:
df['train'][0]

{'answer': 'SELECT COUNT(*) FROM rpt_actual_casos WHERE EXTRACT(YEAR FROM fechaTicket) = 2025',
 'question': '¿Cuántos tickets se han generado en el año 2025?',
 'context': 'CREATE TABLE rpt_actual_casos (\n            IdCaso INT NOT NULL,\n            idTicket BIGINT NULL,\n            idCliente BIGINT NULL,\n            canal VARCHAR(45) NULL,\n            estadoTicket INT NULL,\n            Estado VARCHAR(50) NULL,\n            frecepcion TIMESTAMP NULL,\n            fgestion TIMESTAMP NULL,\n            fechaEjecutivo TIMESTAMP NULL,\n            fechaTicket TIMESTAMP NULL,\n            fechaCierreRequerimiento TIMESTAMP NULL,\n            tiempoRespuesta_min INT NULL,\n            nombre_area VARCHAR(200) NULL,\n            nombre_categoria VARCHAR(500) NULL,\n            descripcionProceso VARCHAR(100) NULL,\n            descripcionSubProceso VARCHAR(100) NULL,\n            mensajeTicket TEXT NULL,\n            scriptFase NVARCHAR(100) NULL,\n            fechaAsignacion TIMESTAMP

In [10]:
create_sql_prompt(df["train"][0])

{'text': '### Instrucciones:\nEres un modelo especializado en convertir lenguaje natural a SQL.Dada una pregunta en lenguaje natural y un contexto.Responde solo **solo con la consulta SQL** correspondiente.\n\n### Pregunta:\n¿Cuántos tickets se han generado en el año 2025?\n\n### Contexto:\n\nCREATE TABLE rpt_actual_casos (\n\tIdCaso INT NOT NULL,\n    idTicket BIGINT NULL,\n    idCliente BIGINT NULL,\n    canal VARCHAR(45) NULL,\n    estadoTicket INT NULL,\n    Estado VARCHAR(50) NULL,\n    frecepcion TIMESTAMP NULL,\n    fgestion TIMESTAMP NULL,\n    fechaEjecutivo TIMESTAMP NULL,\n    fechaTicket TIMESTAMP NULL,\n    fechaCierreRequerimiento TIMESTAMP NULL,\n    tiempoRespuesta_min INT NULL,\n    nombre_area VARCHAR(200) NULL,\n    nombre_categoria VARCHAR(500) NULL,\n    descripcionProceso VARCHAR(100) NULL,\n    descripcionSubProceso VARCHAR(100) NULL,\n    mensajeTicket TEXT NULL,\n    scriptFase NVARCHAR(100) NULL,\n    fechaAsignacion TIMESTAMP NULL,\n    fechaResolucion TIMEST

In [11]:
sql_dataset = df.map(create_sql_prompt)

Map:   0%|          | 0/64000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [12]:
sql_dataset['train'][0]

{'answer': 'SELECT COUNT(*) FROM rpt_actual_casos WHERE EXTRACT(YEAR FROM fechaTicket) = 2025',
 'question': '¿Cuántos tickets se han generado en el año 2025?',
 'context': 'CREATE TABLE rpt_actual_casos (\n            IdCaso INT NOT NULL,\n            idTicket BIGINT NULL,\n            idCliente BIGINT NULL,\n            canal VARCHAR(45) NULL,\n            estadoTicket INT NULL,\n            Estado VARCHAR(50) NULL,\n            frecepcion TIMESTAMP NULL,\n            fgestion TIMESTAMP NULL,\n            fechaEjecutivo TIMESTAMP NULL,\n            fechaTicket TIMESTAMP NULL,\n            fechaCierreRequerimiento TIMESTAMP NULL,\n            tiempoRespuesta_min INT NULL,\n            nombre_area VARCHAR(200) NULL,\n            nombre_categoria VARCHAR(500) NULL,\n            descripcionProceso VARCHAR(100) NULL,\n            descripcionSubProceso VARCHAR(100) NULL,\n            mensajeTicket TEXT NULL,\n            scriptFase NVARCHAR(100) NULL,\n            fechaAsignacion TIMESTAMP

#### Cargar el modelo a afinar y preprocesarlo

Usaremos mbart-large-50, que es multilingüe y T5 realizaremos fine-tuning en tareas de conversión de texto a SQL

In [13]:
from transformers import AutoTokenizer,MBartForConditionalGeneration
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [14]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

MBartScaledWordEmbedding(250054, 1024, padding_idx=1)

In [15]:
from transformers import set_seed, GenerationConfig

class Generador:
    def __init__(self, model, tokenizer, seed=42):
        self.TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
            ### Instrucciones:
            {system_message}

            ### Pregunta:
            {input}

            ### Contexto:
            {context}

            ### Respuesta:
            <SQL>
            </SQL>
        """
        self.SYSTEM_MESSAGE = (
            "Eres un modelo especializado en convertir lenguaje natural a SQL."
            "Dada una pregunta en lenguaje natural y un contexto."
            "Responde solo **solo con la consulta SQL** correspondiente."
        )
        self.context="""
        CREATE TABLE rpt_actual_casos (
        IdCaso INT NOT NULL,
          idTicket BIGINT NULL,
          idCliente BIGINT NULL,
          canal VARCHAR(45) NULL,
          estadoTicket INT NULL,
          Estado VARCHAR(50) NULL,
          frecepcion TIMESTAMP NULL,
          fgestion TIMESTAMP NULL,
          fechaEjecutivo TIMESTAMP NULL,
          fechaTicket TIMESTAMP NULL,
          fechaCierreRequerimiento TIMESTAMP NULL,
          tiempoRespuesta_min INT NULL,
          nombre_area VARCHAR(200) NULL,
          nombre_categoria VARCHAR(500) NULL,
          descripcionProceso VARCHAR(100) NULL,
          descripcionSubProceso VARCHAR(100) NULL,
          mensajeTicket TEXT NULL,
          scriptFase NVARCHAR(100) NULL,
          fechaAsignacion TIMESTAMP NULL,
          fechaResolucion TIMESTAMP NULL,
          mensajeResolucion TEXT NULL,
          nombreCliente VARCHAR(136) NOT NULL,
          descripcionGrupo VARCHAR(100) NULL,
          PNR VARCHAR(50) NULL,
          Usuario_Creacion VARCHAR(50) NULL,
          Usuario_Cierre VARCHAR(50) NULL
      );
        """
        set_seed(seed)
        self.model = model
        self.tokenizer = tokenizer


    def get_generation_config(self, max_tokens=1024, top_k=1024, temperature=1e-4):
        return GenerationConfig(
            max_new_tokens=max_tokens,
            do_sample=True,
            top_k=top_k,
            temperature=temperature,
            eos_token_id=self.model.config.eos_token_id,
        )

    def generate_sample(self, sample):
      prompt_package = self.create_sql_prompt_and_response(sample)
      config = self.get_generation_config()


      inputs = self.tokenizer(prompt_package["full_prompt"], return_tensors="pt").to("cuda")


      self.model.to("cuda")


      output = self.model.generate(
          **inputs,
          max_new_tokens=config.max_new_tokens,
          do_sample=True,
          top_k=config.top_k,
          top_p=0.95,
          temperature=0.3,
          repetition_penalty=1.2,
          eos_token_id=self.model.config.eos_token_id
      )

      generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)


      if "<SQL>" in generated_text:
          generated_text = generated_text.replace("<SQL>", "").replace("</SQL>", "").strip()

      return {
          "question": sample["question"],
          "ground_truth": prompt_package["ground_truth"],
          "model_response": generated_text
      }



    def create_sql_prompt_and_response(self, sample):
        full_prompt = self.TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
            system_message=self.SYSTEM_MESSAGE,
            input=sample["question"],
            context=self.context
        )

        return {
            "full_prompt": full_prompt,
            "ground_truth": sample["answer"]
        }


In [16]:
import json

generador = Generador(model, tokenizer)
resultado = generador.generate_sample(sql_dataset['test'][10])
print(json.dumps(resultado, indent=4, ensure_ascii=False))

{
    "question": "¿Cuántos casos hay con el estado 'En Proceso' en el área 'Environmental consultant'?",
    "ground_truth": "SELECT COUNT(*) FROM rpt_actual_casos WHERE Estado = 'En Proceso' AND nombre_area = 'Environmental consultant'",
    "model_response": "### Instrucciones: Eres un modelo especializado en convertir lenguaje natural a SQL.Dada una pregunta en lenguaje natural y un contexto.Responde solo **solo con la consulta SQL** correspondiente. ### Pregunta: ¿Cuántos casos hay con el estado 'En Proceso' en el área 'Environmental consultant'? ### Contexto: CREATE TABLE rpt_actual_casos ( IdCaso INT NOT NULL, idTicket BIGINT NULL, idCliente BIGINT NULL, canal VARCHAR(45) NULL, estadoTicket INT NULL, Estado VARCHAR(50) NULL, frecepcion TIMESTAMP NULL, fgestion TIMESTAMP NULL, fechaEjecutivo TIMESTAMP NULL, fechaTicket TIMESTAMP NULL, fechaCierreRequerimiento TIMESTAMP NULL, tiempoRespuesta_min INT NULL, nombre_area VARCHAR(200) NULL, nombre_categoria VARCHAR(500) NULL, descripci

In [17]:
from transformers import TrainingArguments
from trl import SFTTrainer
training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing=False,
 max_grad_norm= 0.3,
 ##num_train_epochs=5,
 max_steps=500,
 learning_rate=5e-5,
 save_total_limit=3,
 logging_steps=50,
 weight_decay=0.1,
 output_dir="/content/drive/MyDrive/IA/outputs/model/t5_text2sql_model_v1",
 optim="adamw_torch",
 lr_scheduler_type="cosine",
 eval_strategy="steps",
 eval_steps=500,
 warmup_ratio=0.05,
 fp16=False,
 report_to="none"
)

In [18]:
def preprocess_function(reg):
    return {"text": reg["text"]}


train_data = sql_dataset["train"].map(preprocess_function, remove_columns=["answer", "question", "context"])
eval_data = sql_dataset["validation"].map(preprocess_function, remove_columns=["answer", "question", "context"])


Map:   0%|          | 0/64000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [19]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    processing_class=tokenizer,
    args=training_args,

)




Converting train dataset to ChatML:   0%|          | 0/64000 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/64000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/64000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/64000 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/8000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [20]:
!pip install bitsandbytes

In [21]:
model.config.use_cache = False

In [22]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
trainer.optimizer = optimizer

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [24]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.000100,0.000011


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.15954513359977865, metrics={'train_runtime': 1251.6586, 'train_samples_per_second': 6.392, 'train_steps_per_second': 0.399, 'total_flos': 7365676209733632.0, 'train_loss': 0.15954513359977865})

In [25]:
trainer.save_model("/content/drive/MyDrive/IA/outputs/model/t5_text2sql_model_v1")
tokenizer.save_pretrained("/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer")

('/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer/sentencepiece.bpe.model',
 '/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer/added_tokens.json',
 '/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer/tokenizer.json')

In [26]:
from transformers import MBartForConditionalGeneration, AutoTokenizer

# Directorios de modelo y tokenizer
model_dir = "/content/drive/MyDrive/IA/outputs/model/t5_text2sql_model_v1"
tokenizer_dir = "/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer"

# Cargar el modelo y el tokenizer
model_ = MBartForConditionalGeneration.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Mover el modelo a GPU si está disponible
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=

In [27]:
import json

generador_ = Generador(model_, tokenizer)
resultado = generador_.generate_sample(sql_dataset['test'][10])
print(json.dumps(resultado, indent=4, ensure_ascii=False))

{
    "question": "¿Cuántos casos hay con el estado 'En Proceso' en el área 'Environmental consultant'?",
    "ground_truth": "SELECT COUNT(*) FROM rpt_actual_casos WHERE Estado = 'En Proceso' AND nombre_area = 'Environmental consultant'",
    "model_response": "### Instrucciones: Eres un modelo especializado en convertir lenguaje natural a SQL.Dada una pregunta en lenguaje natural y un contexto.Responde solo **solo con la consulta SQL** correspondiente. ### Pregunta: ¿Cuántos casos hay con el estado 'En Proceso' en el área 'Environmental consultant'? ### Contexto: CREATE TABLE rpt_actual_casos ( IdCaso INT NOT NULL, idTicket BIGINT NULL, idCliente BIGINT NULL, canal VARCHAR(45) NULL, estadoTicket INT NULL, Estado VARCHAR(50) NULL, frecepcion TIMESTAMP NULL, fgestion TIMESTAMP NULL, fechaEjecutivo TIMESTAMP NULL, fechaTicket TIMESTAMP NULL, fechaCierreRequerimiento TIMESTAMP NULL, tiempoRespuesta_min INT NULL, nombre_area VARCHAR(200) NULL, nombre_categoria VARCHAR(500) NULL, descripci